In [ ]:
!pip install bitsandbytes

In [ ]:
import pandas as pd
import torch
from torch import bfloat16
import re
import bitsandbytes
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from nltk.translate.gleu_score import sentence_gleu


In [ ]:
data = pd.read_excel('/content/tu_impersonal_ejemplos_chatGPT.xlsx')

In [ ]:
data = data.rename(columns={
    'Frase Incorrecta (Tú Impersonal)': 'input',
    'Frase Correcta (Se Impersonal)': 'target'
})

data['input'] = data['input'].str.lower()
data['target'] = data['target'].str.lower()

def eliminar_espacios_innecesarios(texto):
    texto = texto.strip()
    texto = re.sub(r'\s{2,}', ' ', texto)
    return texto

def anadir_punto_final(texto):
    texto = texto.strip()
    if not texto.endswith('.'):
        texto += '.'
    return texto

data['input'] = data['input'].apply(anadir_punto_final)
data['target'] = data['target'].apply(anadir_punto_final)
data['input'] = data['input'].apply(eliminar_espacios_innecesarios)
data['target'] = data['target'].apply(eliminar_espacios_innecesarios)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

nf4_config = transformers.BitsAndBytesConfig(
 load_in_4bit=True,
 bnb_4bit_quant_type='nf4',
 bnb_4bit_use_double_quant=True,
 bnb_4bit_compute_dtype=bfloat16
)

In [ ]:
!huggingface-cli login

In [ ]:
model_id = 'mistralai/Mistral-7B-Instruct-v0.3'

tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

model = transformers.AutoModelForCausalLM.from_pretrained(
 model_id,
 trust_remote_code=True,
 quantization_config=nf4_config,
 device_map='auto',
)
model.eval()

In [ ]:
generator = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task='text-generation',
    device_map="auto",
    max_new_tokens = 256,
)

In [ ]:
prompt = """
<s>[INST] <<SYS>>
Eres un experto en corrección de textos en español. Tu tarea es transformar enunciados en segunda persona del singular ("tú") en formulaciones impersonales con "se", adecuando el tiempo verbal, número y persona al contexto, y manteniendo el significado original.
<</SYS>>

Ejemplos:
- Original: Si no estudias, no apruebas los exámenes.
- Corregido: Si no se estudia, no se aprueban los exámenes.
- Original: Durante el proceso de análisis de datos, tú organizas la información en categorías.
- Corregido: Durante el proceso de análisis de datos, se organiza la información en categorías.

Instrucción:
Corrige el siguiente texto siguiendo estas indicaciones y devuelve únicamente el texto reformulado:

[DOCUMENTO]

Devuelve solo el texto corregido.
[/INST]
"""

In [ ]:
resultados = []

for idx, row in data.iterrows():
    frase_original = row['input']
    referencia = row['target']

    prompt_actual = prompt.replace("[DOCUMENTO]", frase_original)

    generado = generator(
        prompt_actual,
        do_sample=False
    )

    salida_modelo = generado[0]['generated_text'].split("[/INST]")[-1].strip()

    ref_tokens = referencia.split()
    gen_tokens = salida_modelo.split()
    gleu = sentence_gleu([ref_tokens], gen_tokens)

    resultados.append({
        "original": frase_original,
        "generada": salida_modelo,
        "referencia": referencia,
        "gleu": gleu
    })

    print(f"Original: {frase_original}")
    print(f"Generada: {salida_modelo}")
    print(f"Referencia: {referencia}")
    print(f"GLEU: {gleu:.4f}")
    print("-" * 60)

df_resultados = pd.DataFrame(resultados)

gleu_medio = df_resultados['gleu'].mean()
print(f"GLEU medio de todas las frases: {gleu_medio:.4f}")

In [ ]:
prompt_feedback = """
<s>[INST] <<SYS>>
Eres experto en retroalimentación lingüística en español. Tu tarea es analizar una frase recibida con error y su corrección. Explica el error y por qué, en contextos formales o escritos, conviene usar formas impersonales con "se" y verbo en tercera persona. Indica la importancia de ajustar tiempo, número y persona para mantener la concordancia. Da un feedback claro y concreto, sin repetir la corrección literal.
<</SYS>>

Ejemplos:
- Original: Si no estudias, no apruebas los exámenes.
- Corregido: Si no se estudia, no se aprueban los exámenes.
- Feedback: En contextos formales se recomienda evitar el "tú" para mantener impersonalidad y objetividad. Por eso se usa la construcción con "se" y el verbo en tercera persona plural, concordando con "exámenes".

Instrucción:
A continuación, se muestra la frase con error y su corrección. Proporciona un feedback específico que explique el error y oriente sobre cómo evitarlo:

Original: [FRASE_CON_ERROR]
Corregido: [FRASE_CORREGIDA]
[/INST]
"""

In [ ]:
resultados = []

for idx, row in data.iterrows():
    frase_original = row['input']
    referencia = row['target']

    prompt_actual = (
        prompt_feedback
        .replace("[FRASE_CON_ERROR]", frase_original)
        .replace("[FRASE_CORREGIDA]", referencia)
    )

    generado = generator(
        prompt_actual,
        max_new_tokens=256,
        do_sample=False
    )

    salida_modelo = generado[0]['generated_text'].split("[/INST]")[-1].strip()

    resultados.append({
        "original": frase_original,
        "corregida": referencia,
        "feedback": salida_modelo,
    })

    print(f"Original (con error): {frase_original}")
    print(f"Corregida: {referencia}")
    print(f"Feedback: {salida_modelo}")
    print("-" * 60)